In [1]:
import time
import datetime
import requests
import json
import pandas as pd

import dash
from dash import dash_table
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline

<function cufflinks.offline.go_offline(connected=None)>

In [2]:
#query = """query {
#    saleAuction(id: 2328) {
#    id
#    endedAt
#    endingPrice
#    }
#}"""

query = """query {
  saleAuctions(first:1000 orderBy: endedAt orderDirection: desc 
  		where: {
        open: false
        purchasePrice_not: null
      }
  		
  
  		) {
    id
    tokenId {
      id
      rarity
      generation
      mainClass
      subClass
      statBoost1
      statBoost2
      profession
      summons
      maxSummons
    }
  	endedAt
    purchasePrice
  }
}
"""

url = "https://graph.defikingdoms.com/subgraphs/name/defikingdoms/apiv5"
r = requests.post(url, json={"query": query})

In [3]:
if r.status_code == 200:
    print(json.dumps(r.json(), indent=2))
else:
    raise Exception(f"Query failed to run with a {r.status_code}.")

{
  "data": {
    "saleAuctions": [
      {
        "endedAt": "1636225815",
        "id": "15047",
        "purchasePrice": "179000000000000000000",
        "tokenId": {
          "generation": 3,
          "id": "10857",
          "mainClass": "Thief",
          "maxSummons": 2,
          "profession": "fishing",
          "rarity": 1,
          "statBoost1": "END",
          "statBoost2": "DEX",
          "subClass": "Summoner",
          "summons": 1
        }
      },
      {
        "endedAt": "1636225568",
        "id": "15075",
        "purchasePrice": "165000000000000000000",
        "tokenId": {
          "generation": 2,
          "id": "12991",
          "mainClass": "Thief",
          "maxSummons": 8,
          "profession": "foraging",
          "rarity": 0,
          "statBoost1": "VIT",
          "statBoost2": "INT",
          "subClass": "Wizard",
          "summons": 0
        }
      },
      {
        "endedAt": "1636225488",
        "id": "14185",
        "purchase

In [4]:
json_data = json.loads(r.text)

currentTime = datetime.datetime.utcnow()

In [5]:
df_data = json_data['data']['saleAuctions']
df = pd.DataFrame(df_data)

In [6]:
df.head()

,endedAt,id,purchasePrice,tokenId
0,1636225815,15047,179000000000000000000,"{'generation': 3, 'id': '10857', 'mainClass': ..."
1,1636225568,15075,165000000000000000000,"{'generation': 2, 'id': '12991', 'mainClass': ..."
2,1636225488,14185,489000000000000000000,"{'generation': 1, 'id': '3362', 'mainClass': '..."
3,1636225148,14818,5499000000000000000000,"{'generation': 0, 'id': '1257', 'mainClass': '..."
4,1636225022,15018,275000000000000000000,"{'generation': 1, 'id': '12953', 'mainClass': ..."


In [7]:
df['tokenId'].apply(pd.Series)

,generation,id,mainClass,maxSummons,profession,rarity,statBoost1,statBoost2,subClass,summons
0,3,10857,Thief,2,fishing,1,END,DEX,Summoner,1
1,2,12991,Thief,8,foraging,0,VIT,INT,Wizard,0
2,1,3362,Knight,10,mining,0,AGI,INT,Thief,1
3,0,1257,Wizard,11,gardening,2,AGI,INT,Archer,8
4,1,12953,Priest,10,fishing,1,STR,AGI,Knight,0
...,...,...,...,...,...,...,...,...,...,...
995,2,10827,Pirate,9,foraging,0,VIT,VIT,Warrior,2
996,2,10838,Priest,7,mining,0,DEX,VIT,Warrior,0
997,2,10754,Pirate,9,fishing,0,DEX,INT,Wizard,0
998,1,2700,Pirate,10,fishing,0,STR,DEX,Warrior,2


In [8]:
df2 = df['tokenId'].apply(pd.Series)

In [9]:
df2 = pd.concat([df2, df['purchasePrice']], axis=1)
df2 = pd.concat([df2, df['endedAt']], axis=1)
df2.head()

,generation,id,mainClass,maxSummons,profession,rarity,statBoost1,statBoost2,subClass,summons,purchasePrice,endedAt
0,3,10857,Thief,2,fishing,1,END,DEX,Summoner,1,179000000000000000000,1636225815
1,2,12991,Thief,8,foraging,0,VIT,INT,Wizard,0,165000000000000000000,1636225568
2,1,3362,Knight,10,mining,0,AGI,INT,Thief,1,489000000000000000000,1636225488
3,0,1257,Wizard,11,gardening,2,AGI,INT,Archer,8,5499000000000000000000,1636225148
4,1,12953,Priest,10,fishing,1,STR,AGI,Knight,0,275000000000000000000,1636225022


In [10]:
#df2 = pd.concat([df['tokenId'].apply(pd.Series), df['purchasePrice']], axis=1)
#df2 = pd.concat([df['tokenId'].apply(pd.Series), df['endedAt']], axis=1)

In [11]:
cols = ['id', 'rarity', 'generation', 'mainClass', 'subClass', 'statBoost1', 'statBoost2', 'profession', 'summons', 'maxSummons', 'purchasePrice', 'endedAt']

In [12]:
df2 = df2.reindex(columns=cols)

In [13]:
df2['rarity'] = df2['rarity'].replace([0, 1, 2, 3, 4], ['common', 'uncommon', 'rare', 'legendary', 'mythic'])
#print(df2)

In [14]:
df2.head()

,id,rarity,generation,mainClass,subClass,statBoost1,statBoost2,profession,summons,maxSummons,purchasePrice,endedAt
0,10857,uncommon,3,Thief,Summoner,END,DEX,fishing,1,2,179000000000000000000,1636225815
1,12991,common,2,Thief,Wizard,VIT,INT,foraging,0,8,165000000000000000000,1636225568
2,3362,common,1,Knight,Thief,AGI,INT,mining,1,10,489000000000000000000,1636225488
3,1257,rare,0,Wizard,Archer,AGI,INT,gardening,8,11,5499000000000000000000,1636225148
4,12953,uncommon,1,Priest,Knight,STR,AGI,fishing,0,10,275000000000000000000,1636225022


In [15]:
#drop empty values from purchasePrice

df2.dropna(subset=['purchasePrice'])

,id,rarity,generation,mainClass,subClass,statBoost1,statBoost2,profession,summons,maxSummons,purchasePrice,endedAt
0,10857,uncommon,3,Thief,Summoner,END,DEX,fishing,1,2,179000000000000000000,1636225815
1,12991,common,2,Thief,Wizard,VIT,INT,foraging,0,8,165000000000000000000,1636225568
2,3362,common,1,Knight,Thief,AGI,INT,mining,1,10,489000000000000000000,1636225488
3,1257,rare,0,Wizard,Archer,AGI,INT,gardening,8,11,5499000000000000000000,1636225148
4,12953,uncommon,1,Priest,Knight,STR,AGI,fishing,0,10,275000000000000000000,1636225022
...,...,...,...,...,...,...,...,...,...,...,...,...
995,10827,common,2,Pirate,Warrior,VIT,VIT,foraging,2,9,180000000000000000000,1636113952
996,10838,common,2,Priest,Warrior,DEX,VIT,mining,0,7,149000000000000000000,1636113843
997,10754,common,2,Pirate,Wizard,DEX,INT,fishing,0,9,150000000000000000000,1636113790
998,2700,common,1,Pirate,Warrior,STR,DEX,fishing,2,10,199000000000000000000,1636113775


In [16]:
soldPrice = []

for x in df['purchasePrice']:
    for y in x:
        priceLen = len(x)-16
    x = x[: priceLen]
    x = int(float(x))/100
    soldPrice.append(x)


In [17]:
df2['soldPrice'] = soldPrice
df2 = df2.drop(['purchasePrice'], axis=1)

In [18]:
df2.head()

,id,rarity,generation,mainClass,subClass,statBoost1,statBoost2,profession,summons,maxSummons,endedAt,soldPrice
0,10857,uncommon,3,Thief,Summoner,END,DEX,fishing,1,2,1636225815,179.0
1,12991,common,2,Thief,Wizard,VIT,INT,foraging,0,8,1636225568,165.0
2,3362,common,1,Knight,Thief,AGI,INT,mining,1,10,1636225488,489.0
3,1257,rare,0,Wizard,Archer,AGI,INT,gardening,8,11,1636225148,5499.0
4,12953,uncommon,1,Priest,Knight,STR,AGI,fishing,0,10,1636225022,275.0


In [19]:
genstr = []

for x in df2['generation']:
    x = str(x)
    genstr.append(x)
    
df2['generation'] = genstr

In [20]:
utcTime = []

for x in df['endedAt']:
    x = int(x)
    x = datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')
    utcTime.append(x)
    #print(utcTime)
    
df2['timeStamp'] = utcTime
df2 = df2.drop(['endedAt'], axis=1)

In [21]:
#filter

#df2[(df2['generation']==1) & (df2['rarity']=='uncommon')]
#df[(df["a"] > 2) & (df["b"] > 5)]

# gen0 = df2[(df2['generation']==0)]
# gen1 = df2[(df2['generation']==1)]
# gen2 = df2[(df2['generation']==2)]
# gen3 = df2[(df2['generation']==3)]
# gen4 = df2[(df2['generation']==4)]
# genbeyond = df2[(df2['generation']>4)]

knight = df2[(df2['mainClass']=='Knight') & (df2["generation"]=='1')]

#fishing = df2[(df2['profession']=='fishing') & (df2["generation"]=='1')]

#WARRIOR

#warrior = df2[(df2['mainClass']=='Warrior')]

# warriorC = df2[(df2['mainClass']=='Warrior') & (df2["rarity"]=='common')]
# warriorU = df2[(df2['mainClass']=='Warrior') & (df2["rarity"]=='uncommon')]
# warriorR = df2[(df2['mainClass']=='Warrior') & (df2["rarity"]=='rare')]
# warriorL = df2[(df2['mainClass']=='Warrior') & (df2["rarity"]=='legendary')]
# warriorM = df2[(df2['mainClass']=='Warrior') & (df2["rarity"]=='mythic')]


warrior = df2
              
warriorC = df2[(df2["rarity"]=='common')]
warriorU = df2[(df2["rarity"]=='uncommon')]
warriorR = df2[(df2["rarity"]=='rare')]
warriorL = df2[(df2["rarity"]=='legendary')]
warriorM = df2[(df2["rarity"]=='mythic')]


In [22]:
df2.to_excel("tavern_data.xlsx", index=False)

In [23]:
# visualization

warrior.head()

,id,rarity,generation,mainClass,subClass,statBoost1,statBoost2,profession,summons,maxSummons,soldPrice,timeStamp
0,10857,uncommon,3,Thief,Summoner,END,DEX,fishing,1,2,179.0,2021-11-06 14:10:15
1,12991,common,2,Thief,Wizard,VIT,INT,foraging,0,8,165.0,2021-11-06 14:06:08
2,3362,common,1,Knight,Thief,AGI,INT,mining,1,10,489.0,2021-11-06 14:04:48
3,1257,rare,0,Wizard,Archer,AGI,INT,gardening,8,11,5499.0,2021-11-06 13:59:08
4,12953,uncommon,1,Priest,Knight,STR,AGI,fishing,0,10,275.0,2021-11-06 13:57:02


In [24]:
#sns.set_style("whitegrid")

#fig, ax = plt.subplots()
#fig.set_size_inches(15, 15)
#sns.boxplot(x="mainClass", y="soldPrice", data=fishing, hue='rarity', hue_order=['common', 'uncommon', 'rare', 'legendary', 'mythic'], palette='rainbow')

#fig.tight_layout()
#plt.savefig('save_as_a_png.png')

In [25]:


fig = go.Figure()

#px.scatter(warriorC, x="timeStamp", y="soldPrice",
#                 hover_name="id", hover_data={'rarity'})])

fig.add_trace(go.Scatter(x=warriorC.timeStamp, y=warriorC.soldPrice, mode='markers', name='Common',
                        hovertemplate =
    '<b>ID</b>: %{text}<br>' +
    '<b>Price</b>: %{y} Jewels'+
    '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
     text = warriorC['id'] + '<br>' + 
            '<b>Rarity</b>: ' + warriorC['rarity'] + '<br>' + 
            '<b>Generation</b>: ' + warriorC['generation'] + '<br>' + '<br>' + 
            '<b>Main Class</b>: ' + warriorC['mainClass'] + '<br>' +
            '<b>Sub Class</b>: ' + warriorC['subClass'] + '<br>' + 
            '<b>Primary Boost</b>: ' + warriorC['statBoost1'] + '<br>' +
            '<b>Secondary Boost</b>: ' + warriorC['statBoost2'] + '<br>' + 
            '<b>Profession</b>: ' + warriorC['profession']+ '<br>',
     marker = dict(color='rgba(219, 217, 222, 1)', size=7)
     
            ))

fig.add_trace(go.Scatter(x=warriorU.timeStamp, y=warriorU.soldPrice, mode='markers', name='Uncommon',
                        hovertemplate =
    '<b>ID</b>: %{text}<br>' +
    '<b>Price</b>: %{y} Jewels'+
    '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
     text = warriorU['id'] + '<br>' + 
            '<b>Rarity</b>: ' + warriorU['rarity'] + '<br>' +
            '<b>Generation</b>: ' + warriorU['generation'] + '<br>' + '<br>' + 
            '<b>Main Class</b>: ' + warriorU['mainClass'] + '<br>' +
            '<b>Sub Class</b>: ' + warriorU['subClass'] + '<br>' + 
            '<b>Primary Boost</b>: ' + warriorU['statBoost1'] + '<br>' +
            '<b>Secondary Boost</b>: ' + warriorU['statBoost2'] + '<br>' + 
            '<b>Profession</b>: ' + warriorU['profession']+ '<br>',
     marker = dict(color='rgba(115, 191, 131, 1)', size=7)
            ))

fig.add_trace(go.Scatter(x=warriorR.timeStamp, y=warriorR.soldPrice, mode='markers', name='Rare',
                        hovertemplate =
    '<b>ID</b>: %{text}<br>' +
    '<b>Price</b>: %{y} Jewels'+
    '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
     text = warriorR['id'] + '<br>' + 
            '<b>Rarity</b>: ' + warriorR['rarity'] + '<br>' +
            '<b>Generation</b>: ' + warriorR['generation'] + '<br>' + '<br>' + 
            '<b>Main Class</b>: ' + warriorR['mainClass'] + '<br>' +
            '<b>Sub Class</b>: ' + warriorR['subClass'] + '<br>' + 
            '<b>Primary Boost</b>: ' + warriorR['statBoost1'] + '<br>' +
            '<b>Secondary Boost</b>: ' + warriorR['statBoost2'] + '<br>' + 
            '<b>Profession</b>: ' + warriorR['profession']+ '<br>',
    marker = dict(color='rgba(53, 147, 183, 1)', size=7)
            ))

fig.add_trace(go.Scatter(x=warriorL.timeStamp, y=warriorL.soldPrice, mode='markers', name='Legendary',
                        hovertemplate =
    '<b>ID</b>: %{text}<br>' +
    '<b>Price</b>: %{y} Jewels'+
    '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
     text = warriorL['id'] + '<br>' + 
            '<b>Rarity</b>: ' + warriorL['rarity'] + '<br>' +
            '<b>Generation</b>: ' + warriorL['generation'] + '<br>' + '<br>' + 
            '<b>Main Class</b>: ' + warriorL['mainClass'] + '<br>' +
            '<b>Sub Class</b>: ' + warriorL['subClass'] + '<br>' + 
            '<b>Primary Boost</b>: ' + warriorL['statBoost1'] + '<br>' +
            '<b>Secondary Boost</b>: ' + warriorL['statBoost2'] + '<br>' + 
            '<b>Profession</b>: ' + warriorL['profession']+ '<br>',
    marker = dict(color='rgba(255, 164, 62, 1)', size=7)
            ))

fig.add_trace(go.Scatter(x=warriorM.timeStamp, y=warriorM.soldPrice, mode='markers', name='Mythic',
                        hovertemplate =
    '<b>ID</b>: %{text}<br>' +
    '<b>Price</b>: %{y} Jewels'+
    '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
     text = warriorM['id'] + '<br>' + 
            '<b>Rarity</b>: ' + warriorM['rarity'] + '<br>' +
            '<b>Generation</b>: ' + warriorM['generation'] + '<br>' + '<br>' + 
            '<b>Main Class</b>: ' + warriorM['mainClass'] + '<br>' +
            '<b>Sub Class</b>: ' + warriorM['subClass'] + '<br>' + 
            '<b>Primary Boost</b>: ' + warriorM['statBoost1'] + '<br>' +
            '<b>Secondary Boost</b>: ' + warriorM['statBoost2'] + '<br>' + 
            '<b>Profession</b>: ' + warriorM['profession']+ '<br>',
    marker = dict(color='rgba(178, 109, 216, 1)', size=7)          
            ))

fig.update_traces(marker=dict(line=dict(width=.5)))
fig.update_layout(title='Tavern Sales - Last 1000 Heroes Sold', 
                  titlefont=dict(family='Arial', size=24),
                  xaxis=dict(showgrid=True, ticks='outside'), 
                  xaxis_title='Date in UTC',
                  yaxis_title='Jewel',
                  plot_bgcolor='white'
                 )

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)


In [26]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(x=warriorC.timeStamp, y=warriorC.soldPrice, mode='markers', name='Common',
#                         hovertemplate =
#     '<b>ID</b>: %{text}<br>' +
#     '<b>Price</b>: %{y} Jewels'+
#     '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
#      text = warriorC['id'] + '<br>' + 
#             '<b>Rarity</b>: ' + warriorC['rarity'] + '<br>' + 
#             '<b>Generation</b>: ' + warriorC['generation'] + '<br>' + '<br>' + 
#             '<b>Main Class</b>: ' + warriorC['mainClass'] + '<br>' +
#             '<b>Sub Class</b>: ' + warriorC['subClass'] + '<br>' + 
#             '<b>Primary Boost</b>: ' + warriorC['statBoost1'] + '<br>' +
#             '<b>Secondary Boost</b>: ' + warriorC['statBoost2'] + '<br>' + 
#             '<b>Profession</b>: ' + warriorC['profession']+ '<br>',
#      marker = dict(color='rgba(219, 217, 222, 1)', size=7)
     
#             ))

# fig.add_trace(go.Scatter(x=warriorU.timeStamp, y=warriorU.soldPrice, mode='markers', name='Uncommon',
#                         hovertemplate =
#     '<b>ID</b>: %{text}<br>' +
#     '<b>Price</b>: %{y} Jewels'+
#     '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
#      text = warriorU['id'] + '<br>' + 
#             '<b>Rarity</b>: ' + warriorU['rarity'] + '<br>' +
#             '<b>Generation</b>: ' + warriorU['generation'] + '<br>' + '<br>' + 
#             '<b>Main Class</b>: ' + warriorU['mainClass'] + '<br>' +
#             '<b>Sub Class</b>: ' + warriorU['subClass'] + '<br>' + 
#             '<b>Primary Boost</b>: ' + warriorU['statBoost1'] + '<br>' +
#             '<b>Secondary Boost</b>: ' + warriorU['statBoost2'] + '<br>' + 
#             '<b>Profession</b>: ' + warriorU['profession']+ '<br>',
#      marker = dict(color='rgba(115, 191, 131, 1)', size=7)
#             ))

# fig.add_trace(go.Scatter(x=warriorR.timeStamp, y=warriorR.soldPrice, mode='markers', name='Rare',
#                         hovertemplate =
#     '<b>ID</b>: %{text}<br>' +
#     '<b>Price</b>: %{y} Jewels'+
#     '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
#      text = warriorR['id'] + '<br>' + 
#             '<b>Rarity</b>: ' + warriorR['rarity'] + '<br>' +
#             '<b>Generation</b>: ' + warriorR['generation'] + '<br>' + '<br>' + 
#             '<b>Main Class</b>: ' + warriorR['mainClass'] + '<br>' +
#             '<b>Sub Class</b>: ' + warriorR['subClass'] + '<br>' + 
#             '<b>Primary Boost</b>: ' + warriorR['statBoost1'] + '<br>' +
#             '<b>Secondary Boost</b>: ' + warriorR['statBoost2'] + '<br>' + 
#             '<b>Profession</b>: ' + warriorR['profession']+ '<br>',
#     marker = dict(color='rgba(53, 147, 183, 1)', size=7)
#             ))

# fig.add_trace(go.Scatter(x=warriorL.timeStamp, y=warriorL.soldPrice, mode='markers', name='Legendary',
#                         hovertemplate =
#     '<b>ID</b>: %{text}<br>' +
#     '<b>Price</b>: %{y} Jewels'+
#     '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
#      text = warriorL['id'] + '<br>' + 
#             '<b>Rarity</b>: ' + warriorL['rarity'] + '<br>' +
#             '<b>Generation</b>: ' + warriorL['generation'] + '<br>' + '<br>' + 
#             '<b>Main Class</b>: ' + warriorL['mainClass'] + '<br>' +
#             '<b>Sub Class</b>: ' + warriorL['subClass'] + '<br>' + 
#             '<b>Primary Boost</b>: ' + warriorL['statBoost1'] + '<br>' +
#             '<b>Secondary Boost</b>: ' + warriorL['statBoost2'] + '<br>' + 
#             '<b>Profession</b>: ' + warriorL['profession']+ '<br>',
#     marker = dict(color='rgba(255, 164, 62, 1)', size=7)
#             ))

# fig.add_trace(go.Scatter(x=warriorM.timeStamp, y=warriorM.soldPrice, mode='markers', name='Mythic',
#                         hovertemplate =
#     '<b>ID</b>: %{text}<br>' +
#     '<b>Price</b>: %{y} Jewels'+
#     '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
#      text = warriorM['id'] + '<br>' + 
#             '<b>Rarity</b>: ' + warriorM['rarity'] + '<br>' +
#             '<b>Generation</b>: ' + warriorM['generation'] + '<br>' + '<br>' + 
#             '<b>Main Class</b>: ' + warriorM['mainClass'] + '<br>' +
#             '<b>Sub Class</b>: ' + warriorM['subClass'] + '<br>' + 
#             '<b>Primary Boost</b>: ' + warriorM['statBoost1'] + '<br>' +
#             '<b>Secondary Boost</b>: ' + warriorM['statBoost2'] + '<br>' + 
#             '<b>Profession</b>: ' + warriorM['profession']+ '<br>',
#     marker = dict(color='rgba(178, 109, 216, 1)', size=7)          
#             ))

# fig.update_traces(marker=dict(line=dict(width=.5)))
# fig.update_layout(title='Latest Warriors Sold Based on Last 1000 Tavern Sales', 
#                   titlefont=dict(family='Arial', size=24),
#                   xaxis=dict(showgrid=True, ticks='outside'), 
#                   xaxis_title='Date in UTC',
#                   yaxis_title='Jewel',
#                   plot_bgcolor='white'
#                  )

# fig.update_xaxes(showspikes=True)
# fig.update_yaxes(showspikes=True)

#                   #plot_bgcolor='white')

In [27]:
# sns.set_theme()
# # fig, ax = plt.subplots()
# fig.set_size_inches(8, 8)
# sns.boxplot(x="profession", y="soldPrice", data=knight, hue='rarity', hue_order=['common', 'uncommon', 'rare', 'legendary', 'mythic'], palette='rainbow')

In [28]:

#         dash_table.DataTable(
#             id='test_table',
#             columns=[
#             {'name': 'ID', 'id': 'id'},
#             {'name': 'Rarity', 'id': 'rarity'},
#             {'name': 'Generation', 'id': 'generation'},
#             {'name': 'Main Class', 'id': 'mainClass'},
#             {'name': 'Sub Class', 'id': 'subClass'},
#             {'name': 'Primary Stat', 'id': 'statBoost1'},
#             {'name': 'Secondary Stat', 'id': 'statBoost2'},
#             {'name': 'Profession', 'id': 'profession'},
#             {'name': 'Summons Used', 'id': 'summons'},
#             {'name': 'Max Summons', 'id': 'maxSummons'},
#             {'name': 'Sold Price', 'id': 'soldPrice'},
#             {'name': 'Timestamp', 'id': 'timeStamp'}
#             , data=[warrior]]
        

In [ ]:
#Initialize
# Setup the style from the link:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# Embed the style to the dashabord:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
#server = app.server

app.layout = html.Div(
    children=[
        html.H1(children="DeFi Kingdom Tavern Dashboards",),
        html.Div(
            children="Random playground for various tavern dashboards."),
            
        html.Div(
            children="Data last updated: {}.".format(currentTime)),

        html.Div(
            children=[
                html.Div(children = 'Main Class', style={'fontSize': "16px", 'width': '50%'},className = 'menu-title'),
                dcc.Dropdown(
                    id = 'main-class',
                    options = [
                        {'label': MainClass, 'value':MainClass}
                        for MainClass in warrior.mainClass.unique()
                    ], #'warrior' is the filter
                    value ='(All)',
                    clearable = True,
                    searchable = False,
                    className = 'dropdown', style={'fontSize': "14px",'textAlign': 'center'},
                ),
            ],
            className = 'menu',
        ), #the dropdown function
        
        dcc.Graph(id = 'main-chart', figure=fig),
        
        dash_table.DataTable(id='main-table',
        columns=[{"name": i, "id": i} for i in warrior.columns],
        data=warrior.to_dict('records'))
                            
    ]
)

if __name__ == "__main__":
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Nov/2021 14:27:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:27:42] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:27:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:27:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:27:42] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:27:42] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:27:42] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:27:42] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:27:42] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2021 14:27:42] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -


In [ ]:
@app.callback(
    [Output("main-table", "data")],
    [Input(component_id="main-class", component_property="value")]
)
def update_tables(mclass):
    filtered_df = warrior[warrior.mainClass==mclass]
    return[filtered_df.to_dict('records')]

#     warrior = warrior[(warrior['mainClass']==MainClass)]
#     tab_update = dash_table.DataTable( 
#                  columns=[{"name": i, "id": i} for i in warrior.columns],
#                  data=warrior.to_dict('records'))

#     return tab_update

In [31]:
@app.callback(
    [Output("main-chart", "figure")],
    [
        Input("main-class", "value"),
#         Input("type-filter", "value"),
#         Input("date-range", "start_date"),
#         Input("date-range", "end_date"),
    ],
)
def update_charts(MClass):
    filtered_dataC = warrior[(warrior['mainClass'] == MClass) & (warrior["rarity"]=='common')]
    filtered_dataU = warrior[(warrior['MainClass'] == MClass) & (warrior["rarity"]=='uncommon')] 
    filtered_dataR = warrior[(warrior['MainClass'] == MClass) & (warrior["rarity"]=='rare')] 
    filtered_dataL = warrior[(warrior['MainClass'] == MClass) & (warrior["rarity"]=='legendary')] 
    filtered_dataM = warrior[(warrior['MainClass'] == MClass) & (warrior["rarity"]=='mythic')] 

fig_update = go.Figure()

fig_update.add_trace(go.Scatter(x=filtered_dataC.timeStamp, y=filtered_dataC.soldPrice, mode='markers', name='Common',
                        hovertemplate =
    '<b>ID</b>: %{text}<br>' +
    '<b>Price</b>: %{y} Jewels'+
    '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
     text = filtered_dataC['id'] + '<br>' + 
            '<b>Rarity</b>: ' + filtered_dataC['rarity'] + '<br>' + 
            '<b>Generation</b>: ' + filtered_dataC['generation'] + '<br>' + '<br>' + 
            '<b>Main Class</b>: ' + filtered_dataC['mainClass'] + '<br>' +
            '<b>Sub Class</b>: ' + filtered_dataC['subClass'] + '<br>' + 
            '<b>Primary Boost</b>: ' + filtered_dataC['statBoost1'] + '<br>' +
            '<b>Secondary Boost</b>: ' + filtered_dataC['statBoost2'] + '<br>' + 
            '<b>Profession</b>: ' + filtered_dataC['profession']+ '<br>',
     marker = dict(color='rgba(219, 217, 222, 1)', size=7)
     
            ))

fig_update.add_trace(go.Scatter(x=filtered_dataU.timeStamp, y=filtered_dataU.soldPrice, mode='markers', name='Uncommon',
                        hovertemplate =
    '<b>ID</b>: %{text}<br>' +
    '<b>Price</b>: %{y} Jewels'+
    '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
     text = filtered_dataU['id'] + '<br>' + 
            '<b>Rarity</b>: ' + filtered_dataU['rarity'] + '<br>' +
            '<b>Generation</b>: ' + filtered_dataU['generation'] + '<br>' + '<br>' + 
            '<b>Main Class</b>: ' + filtered_dataU['mainClass'] + '<br>' +
            '<b>Sub Class</b>: ' + filtered_dataU['subClass'] + '<br>' + 
            '<b>Primary Boost</b>: ' + filtered_dataU['statBoost1'] + '<br>' +
            '<b>Secondary Boost</b>: ' + filtered_dataU['statBoost2'] + '<br>' + 
            '<b>Profession</b>: ' + filtered_dataU['profession']+ '<br>',
     marker = dict(color='rgba(115, 191, 131, 1)', size=7)
            ))

fig_update.add_trace(go.Scatter(x=filtered_dataR.timeStamp, y=filtered_dataR.soldPrice, mode='markers', name='Rare',
                        hovertemplate =
    '<b>ID</b>: %{text}<br>' +
    '<b>Price</b>: %{y} Jewels'+
    '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
     text = filtered_dataR['id'] + '<br>' + 
            '<b>Rarity</b>: ' + filtered_dataR['rarity'] + '<br>' +
            '<b>Generation</b>: ' + filtered_dataR['generation'] + '<br>' + '<br>' + 
            '<b>Main Class</b>: ' + filtered_dataR['mainClass'] + '<br>' +
            '<b>Sub Class</b>: ' + filtered_dataR['subClass'] + '<br>' + 
            '<b>Primary Boost</b>: ' + filtered_dataR['statBoost1'] + '<br>' +
            '<b>Secondary Boost</b>: ' + filtered_dataR['statBoost2'] + '<br>' + 
            '<b>Profession</b>: ' + filtered_dataR['profession']+ '<br>',
    marker = dict(color='rgba(53, 147, 183, 1)', size=7)
            ))

fig_update.add_trace(go.Scatter(x=filtered_dataL.timeStamp, y=warriorL.soldPrice, mode='markers', name='Legendary',
                        hovertemplate =
    '<b>ID</b>: %{text}<br>' +
    '<b>Price</b>: %{y} Jewels'+
    '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
     text = warriorL['id'] + '<br>' + 
            '<b>Rarity</b>: ' + filtered_dataL['rarity'] + '<br>' +
            '<b>Generation</b>: ' + filtered_dataL['generation'] + '<br>' + '<br>' + 
            '<b>Main Class</b>: ' + filtered_dataL['mainClass'] + '<br>' +
            '<b>Sub Class</b>: ' + filtered_dataL['subClass'] + '<br>' + 
            '<b>Primary Boost</b>: ' + filtered_dataL['statBoost1'] + '<br>' +
            '<b>Secondary Boost</b>: ' + filtered_dataL['statBoost2'] + '<br>' + 
            '<b>Profession</b>: ' + filtered_dataL['profession']+ '<br>',
    marker = dict(color='rgba(255, 164, 62, 1)', size=7)
            ))

fig_update.add_trace(go.Scatter(x=filtered_dataM.timeStamp, y=filtered_dataM.soldPrice, mode='markers', name='Mythic',
                        hovertemplate =
    '<b>ID</b>: %{text}<br>' +
    '<b>Price</b>: %{y} Jewels'+
    '<br><b>Sold At</b>: %{x} UTC<br><extra></extra>',
     text = filtered_dataM['id'] + '<br>' + 
            '<b>Rarity</b>: ' + filtered_dataM['rarity'] + '<br>' +
            '<b>Generation</b>: ' + filtered_dataM['generation'] + '<br>' + '<br>' + 
            '<b>Main Class</b>: ' + filtered_dataM['mainClass'] + '<br>' +
            '<b>Sub Class</b>: ' + filtered_dataM['subClass'] + '<br>' + 
            '<b>Primary Boost</b>: ' + filtered_dataM['statBoost1'] + '<br>' +
            '<b>Secondary Boost</b>: ' + filtered_dataM['statBoost2'] + '<br>' + 
            '<b>Profession</b>: ' + filtered_dataM['profession']+ '<br>',
    marker = dict(color='rgba(178, 109, 216, 1)', size=7)          
            ))

fig_update.update_traces(marker=dict(line=dict(width=.5)))
fig_update.update_layout(title='Tavern Sales - Last 1000 Heroes Sold', 
                  titlefont=dict(family='Arial', size=24),
                  xaxis=dict(showgrid=True, ticks='outside'), 
                  xaxis_title='Date in UTC',
                  yaxis_title='Jewel',
                  plot_bgcolor='white'
                 )

fig_update.update_xaxes(showspikes=True)
fig_update.update_yaxes(showspikes=True)

return fig_update

NameError: name 'filtered_dataC' is not defined